In [1]:
import os
import sys
import csv
import wave
import copy
import math

import numpy as np
import pandas as pd

from sklearn.preprocessing import label_binarize
from sklearn.cross_validation import StratifiedKFold, KFold, train_test_split
from sklearn.svm import OneClassSVM, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop

sys.path.append("../")

from utilities.utils import *

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

from IPython.display import clear_output

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [2]:
batch_size = 64
nb_feat = 34
nb_class = 4
nb_epoch = 80

optimizer = 'Adadelta'

In [3]:
params = Constants()
print(params)

----------------------------------------------------------------------------------------------------
available_emotions            ['ang' 'exc' 'neu' 'sad']                                             
conf_matrix_prefix            iemocap                                                               
framerate                     16000                                                                 
path_to_data                  /home/samarth/emotion_recognition-master/code/utilities/../../data/ses
path_to_features              /home/samarth/emotion_recognition-master/code/utilities/../../data/fea
sessions                      ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']          
types                         {1: <class 'numpy.int8'>, 2: <class 'numpy.int16'>, 4: <class 'numpy.i
----------------------------------------------------------------------------------------------------


In [4]:
path_to_avi = params.path_to_data + 'Session1' + '/dialog/MOCAP_rotated/Ses01F_impro01.txt'

In [5]:
f = open(path_to_avi).read()
f = np.array(f.split('\n'))
f = f[2:]
mocap_rot = []
counter = 0
for data in f:
    mocap_rot.append(data.split(' ')[2:])
print(len(mocap_rot))
print(len(mocap_rot[0]))
print(mocap_rot[0])

13836
165
['-28.03277', '32.01503', '-53.26572', '-0.24866', '22.38171', '-57.92426', '23.89061', '32.98670', '-55.86006', '-23.24082', '35.50025', '98.77892', '0.71984', '23.85440', '80.29511', '27.32115', '37.51287', '96.66002', '51.59958', '59.66769', '-45.87803', '60.92136', '59.61728', '-20.87795', '62.02471', '54.14578', '23.94855', '29.54917', '30.67098', '-18.71798', '43.29584', '34.50971', '-2.27367', '21.20029', '27.47530', '8.28366', '42.79253', '35.78905', '13.05869', '17.40279', '30.54087', '21.57486', '-57.80894', '54.44732', '-44.84093', '-66.45131', '55.02972', '-16.50413', '-64.02947', '48.72470', '26.91160', '-33.81545', '28.04140', '-17.92018', '-48.29249', '32.37606', '-1.73251', '-24.59868', '26.37021', '9.31240', '-43.71247', '32.21557', '14.81166', '-21.03775', '30.50886', '22.34581', '34.57282', '29.54649', '43.73436', '-32.64363', '26.91946', '44.83178', '-1.36057', '18.92686', '39.72273', '0.42066', '11.90556', '21.39331', '13.83638', '18.32322', '0.56940', '0

In [49]:
def get_mocap_rot(path_to_mocap_rot, filename, params=Constants()):
    f = open(path_to_mocap_rot + filename, 'r').read()
    f = np.array(f.split('\n'))
    mocap_rot = []
    mocap_rot_avg = []
    f = f[2:]
    counter = 0
    for data in f:
        counter+=1
        data2 = np.array(data.split(' ')[2:]).astype(np.float)
        if(len(data2)>0):
            mocap_rot_avg.append(data2)
            
    mocap_rot_avg = np.array_split(np.array(mocap_rot_avg), 200)
    for spl in mocap_rot_avg:
        mocap_rot.append(np.mean(spl, axis=0))
    return np.array(mocap_rot)

In [50]:

mocap_rot = get_mocap_rot(params.path_to_data + 'Session1' + '/dialog/MOCAP_rotated/', 'Ses01F_impro01.txt')

In [51]:
mocap_rot.shape


(200, 165)

In [52]:


def read_iemocap_mocap_rot(params=Constants()):
    data = []
    for session in params.sessions:
        path_to_mocap_rot = params.path_to_data + session + '/dialog/MOCAP_rotated/'
        path_to_emotions = params.path_to_data + session + '/dialog/EmoEvaluation/'
        #path_to_transcriptions = params.path_to_data + session + '/dialog/transcriptions/'
        
        files2 = os.listdir(path_to_mocap_rot)
        #print (files2.sort())
        files = []
        for f in files2:
            if f.endswith(".txt"):
                if f[0] == '.':
                    files.append(f[2:-4])
                else:
                    files.append(f[:-4])
                    
        print (session)   
        for f in files:
            print (f)
            mocap_rot = get_mocap_rot(path_to_mocap_rot, f + '.txt')
            emotions = get_emotions(path_to_emotions, f + '.txt')

            for ie, e in enumerate(emotions):
                e.pop("left", None)
                e.pop("right", None)
                e['mocap_rot'] = mocap_rot
                if e['emotion'] in params.available_emotions:
                    data.append(e)
    sort_key = get_field(data, "id")
    return np.array(data)[np.argsort(sort_key)]

In [53]:
data = read_iemocap_mocap_rot(params=params)

Session1
Ses01F_script02_2
Ses01M_script02_1
Ses01F_impro04
Ses01M_impro02
Ses01M_impro05
Ses01F_impro05
Ses01M_impro06
Ses01M_script03_2
Ses01M_impro07
Ses01F_script01_1
Ses01F_script02_1
Ses01M_script03_1
Ses01M_impro04
Ses01M_script03_2
Ses01M_impro03
Ses01F_script02_1
Ses01M_script01_3
Ses01F_impro04
Ses01F_impro03
Ses01M_script01_1
Ses01F_impro02
Ses01F_impro02
Ses01M_impro03
Ses01M_impro01
Ses01F_impro01
Ses01M_script01_2
Ses01F_script03_1
Ses01F_script01_2
Ses01M_script01_1
Ses01M_impro07
Ses01F_impro05
Ses01M_script02_1
Ses01M_impro05
Ses01M_script01_3
Ses01M_script02_2
Ses01F_impro07
Ses01M_impro06
Ses01F_impro07
Ses01F_script01_3
Ses01F_script01_3
Ses01M_script01_2
Ses01M_script02_2
Ses01F_impro01
Ses01F_script02_2
Ses01F_impro06
Ses01M_impro04
Ses01F_script03_2
Ses01F_script03_1
Ses01F_impro03
Ses01F_script03_2
Ses01F_script01_2
Ses01M_script03_1
Ses01F_script01_1
Ses01F_impro06
Ses01M_impro02
Ses01M_impro01
Session2
Ses02F_script01_2
Ses02M_impro08
Ses02F_impro03
Ses02F_scr

In [54]:
import pickle
with open(params.path_to_data + '/../'+'params.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


In [59]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv2D, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution2D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

In [103]:
def build_simple_lstm(nb_feat, nb_class, optimizer='SGD'):
    model = Sequential()
    model.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 165, 1)))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [104]:
model = build_simple_lstm(nb_feat, nb_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 100, 83, 32)       320       
_________________________________________________________________
dropout_35 (Dropout)         (None, 100, 83, 32)       0         
_________________________________________________________________
activation_35 (Activation)   (None, 100, 83, 32)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 50, 42, 64)        18496     
_________________________________________________________________
dropout_36 (Dropout)         (None, 50, 42, 64)        0         
_________________________________________________________________
activation_36 (Activation)   (None, 50, 42, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 25, 21, 64)        36928     
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, padding="same", input_shape=(200, 165,..., strides=(2, 2))`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", strides=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="

In [91]:
Y=[]
for ses_mod in data:
    Y.append(ses_mod['emotion'])
    
Y = to_categorical(Y)

In [92]:
Y.shape

(9824, 4)

In [99]:
x_train = []
from sklearn.preprocessing import normalize

for ses_mod in data:
    x = ses_mod['mocap_rot']
    x_normed = (x - x.min(0)) / x.ptp(0)
    x_normed = x_normed - 0.5
    x_train.append( x_normed )
    


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [100]:
x_train = np.array(x_train)
x_train.shape

(9824, 200, 165)

In [101]:
x_train[0]

array([[ 0.29300938, -0.4291221 ,  0.22139652, ...,  0.25505516,
        -0.28289607,  0.45246834],
       [ 0.28025547, -0.43105038,  0.21967559, ...,  0.26057733,
        -0.27699887,  0.46162557],
       [ 0.28229448, -0.43282538,  0.21574789, ...,  0.26255707,
        -0.28204962,  0.45608459],
       ..., 
       [-0.3041923 ,  0.11176282,  0.00220133, ...,  0.18818161,
         0.40186788,  0.11323058],
       [-0.5       , -0.05297497,  0.14160482, ...,  0.23435174,
         0.48959544, -0.17426309],
       [-0.48213394, -0.03131693,  0.16252626, ...,  0.1620585 ,
         0.5       , -0.22253319]])

In [102]:
x_train = x_train.reshape(-1,200,165,1)
x_train.shape

(9824, 200, 165, 1)

In [105]:
hist = model.fit(x_train, Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 7859 samples, validate on 1965 samples
Epoch 1/30
7859/7859 [==============================] - 14s 2ms/step - loss: 1.3750 - acc: 0.3374 - val_loss: 1.3618 - val_acc: 0.3817
Epoch 2/30
7859/7859 [==============================] - 13s 2ms/step - loss: 1.3664 - acc: 0.3382 - val_loss: 1.3579 - val_acc: 0.3817
Epoch 3/30
4864/7859 [=================>............] - ETA: 4s - loss: 1.3661 - acc: 0.3372

KeyboardInterrupt: 

In [95]:
Y[5000]

array([1, 0, 0, 0])

In [ ]:
hist = model.fit(X_train, y_train, 
                 batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, 
                 validation_data=(X_test, y_test))

In [106]:
path_to_avi = params.path_to_data + 'Session1' + '/dialog/MOCAP_head/Ses01F_impro01.txt'

In [112]:
f = open(path_to_avi).read()
f = np.array(f.split('\n'))
f = f[2:]
mocap_head = []
print(f[0])
print(f[1])
for data in f:
    mocap_head.append(data.split(' ')[2:])
print(len(mocap_head))
print(len(mocap_head[0]))
print(mocap_head[0])

1 0.00834 -7.98901 -3.79535 -16.42346 -52.87054 -130.52053 -19.19184
2 0.01668 -7.99849 -3.79422 -16.41632 -52.90273 -130.55276 -19.21793
13836
6
['-7.98901', '-3.79535', '-16.42346', '-52.87054', '-130.52053', '-19.19184']


In [113]:
def get_mocap_head(path_to_mocap_head, filename, params=Constants()):
    f = open(path_to_mocap_head + filename, 'r').read()
    f = np.array(f.split('\n'))
    mocap_head = []
    mocap_head_avg = []
    f = f[2:]
    counter = 0
    for data in f:
        counter+=1
        data2 = np.array(data.split(' ')[2:]).astype(np.float)
        if(len(data2)>0):
            mocap_head_avg.append(data2)
            
    mocap_head_avg = np.array_split(np.array(mocap_head_avg), 200)
    for spl in mocap_head_avg:
        mocap_head.append(np.mean(spl, axis=0))
    return np.array(mocap_head)

In [154]:
path_to_mocap_head = params.path_to_data + 'Session1' + '/dialog/MOCAP_head/'

p1 = get_mocap_head(path_to_mocap_head, 'Ses01F_impro05.txt')

p1

array([[  -6.91419826,   -1.85966265,  -12.07068167,  -37.72171902,
        -151.00772864,  -20.07986962],
       [  -4.58851553,   -0.30118333,   -6.02337523,  -53.8311622 ,
        -146.74033379,  -13.20579311],
       [   7.1994847 ,    2.57165091,   28.19405811, -153.08642182,
        -143.6798722 ,   23.9247872 ],
       ..., 
       [  13.79968847,   -0.29111893,  -11.53979985,  -36.96312901,
        -150.93543794,   27.45288145],
       [  15.19663641,   -2.04341282,   -8.96844153,  -38.62445702,
        -142.00212374,   34.55065611],
       [  19.96432466,   -2.16521305,   -8.99754145,  -37.51060618,
        -137.48809   ,   52.02116183]])

In [155]:
path_to_mocap_head = params.path_to_data + 'Session1' + '/dialog/MOCAP_head/'

p2 = get_mocap_head(path_to_mocap_head, 'Ses01F_impro04.txt')

p2

array([[  1.05682447,   4.69745569, -10.69277447, -29.03314748,
        -80.98679854,  -6.3053552 ],
       [  2.10166244,   8.03192106,  -5.80895837, -47.15584951,
        -56.99279862,   3.10244634],
       [ 15.98890211,   9.21809146,  -4.72620081, -43.60227382,
        -19.08773203,  49.29260309],
       ..., 
       [ -5.16972221,  -7.26767049, -18.44535713,  10.7866441 ,
        -30.72164689, -22.35441459],
       [-16.44222098,   0.81542828,  -3.3361623 , -37.65814049,
        -43.93689721, -49.17171811],
       [ -8.77459352,  -3.28866311,  -9.63507549, -14.80306582,
        -37.70863311, -37.31502934]])

In [186]:
def read_iemocap_mocap_head(params=Constants()):
    data = []
    ids = {}
    for session in params.sessions:
        path_to_mocap_head = params.path_to_data + session + '/dialog/MOCAP_head/'
        path_to_emotions = params.path_to_data + session + '/dialog/EmoEvaluation/'
        #path_to_transcriptions = params.path_to_data + session + '/dialog/transcriptions/'
        
        files2 = os.listdir(path_to_mocap_head)
        #print (files2.sort())
        files = []
        for f in files2:
            if f.endswith(".txt"):
                if f[0] == '.':
                    files.append(f[2:-4])
                else:
                    files.append(f[:-4])
                    
        print (session)   
        for f in files:
            print (f)
            #mocap_head = get_mocap_head(path_to_mocap_head, f + '.txt')
            emotions = get_emotions(path_to_emotions, f + '.txt')
            #print(mocap_head[:10])
            for ie, e in enumerate(emotions):
                e.pop("left", None)
                e.pop("right", None)
                #e['mocap_head'] = mocap_head
                if e['emotion'] in params.available_emotions:
                    if e['id'] not in ids:
                        e['mocap_head'] = get_mocap_head(path_to_mocap_head, f + '.txt')
                        data.append(e)
                        ids[e['id']] = 1
                        
            break
    sort_key = get_field(data, "id")
    return np.array(data)[np.argsort(sort_key)]

In [187]:
data2 = read_iemocap_mocap_head(params=params)

Session1
Ses01F_script02_2
Session2
Ses02F_script01_2
Session3
Ses03F_script01_3
Session4
Ses04M_impro02
Session5
Ses05F_script02_2


In [188]:
data2[0]

{'a': 1.5,
 'd': 2.5,
 'emo_evo': [['fru'], ['neu'], ['neu']],
 'emotion': 'neu',
 'end': 8.3081,
 'id': 'Ses01F_script02_2_F000',
 'mocap_head': array([[ -7.48291148,   2.12162028,  -7.78480997, -17.49848272,
         -12.30308821, -30.03441803],
        [-10.24596731,   3.68007779,  -6.24543576, -23.34865948,
         -10.0190611 , -34.19517483],
        [ -6.87802783,  -7.19703834, -13.96951352,   6.12203683,
         -17.40342248, -29.75377941],
        ..., 
        [  1.45773962, -12.27991038,  -7.45092104,   2.46887024,
         -26.53674353, -10.02888495],
        [  0.36206118, -10.44756211,  -8.07600308,   4.5343818 ,
         -24.01096038, -13.95601301],
        [  5.01136318,  -2.53337533, -10.45531194,   0.69937439,
         -19.45046564,  -1.03147215]]),
 'start': 5.83,
 'v': 3.0}

In [189]:
data2[1]

{'a': 2.0,
 'd': 3.0,
 'emo_evo': [['ang'], ['neu'], ['neu']],
 'emotion': 'neu',
 'end': 11.4525,
 'id': 'Ses01F_script02_2_F001',
 'mocap_head': array([[ -7.48291148,   2.12162028,  -7.78480997, -17.49848272,
         -12.30308821, -30.03441803],
        [-10.24596731,   3.68007779,  -6.24543576, -23.34865948,
         -10.0190611 , -34.19517483],
        [ -6.87802783,  -7.19703834, -13.96951352,   6.12203683,
         -17.40342248, -29.75377941],
        ..., 
        [  1.45773962, -12.27991038,  -7.45092104,   2.46887024,
         -26.53674353, -10.02888495],
        [  0.36206118, -10.44756211,  -8.07600308,   4.5343818 ,
         -24.01096038, -13.95601301],
        [  5.01136318,  -2.53337533, -10.45531194,   0.69937439,
         -19.45046564,  -1.03147215]]),
 'start': 9.58,
 'v': 2.5}

In [190]:
data2[2]

{'a': 4.5,
 'd': 4.0,
 'emo_evo': [['ang'], ['ang'], ['fru', 'ang']],
 'emotion': 'ang',
 'end': 61.96,
 'id': 'Ses01F_script02_2_F006',
 'mocap_head': array([[ -7.48291148,   2.12162028,  -7.78480997, -17.49848272,
         -12.30308821, -30.03441803],
        [-10.24596731,   3.68007779,  -6.24543576, -23.34865948,
         -10.0190611 , -34.19517483],
        [ -6.87802783,  -7.19703834, -13.96951352,   6.12203683,
         -17.40342248, -29.75377941],
        ..., 
        [  1.45773962, -12.27991038,  -7.45092104,   2.46887024,
         -26.53674353, -10.02888495],
        [  0.36206118, -10.44756211,  -8.07600308,   4.5343818 ,
         -24.01096038, -13.95601301],
        [  5.01136318,  -2.53337533, -10.45531194,   0.69937439,
         -19.45046564,  -1.03147215]]),
 'start': 55.67,
 'v': 1.5}

In [183]:
data2[4]

{'a': 3.5,
 'd': 3.5,
 'emo_evo': [['ang'], ['dis'], ['ang'], ['fru', 'sur', 'ang']],
 'emotion': 'ang',
 'end': 88.02,
 'id': 'Ses01F_impro01_F012',
 'mocap_head': array([[  -7.95062871,   -3.72252457,  -16.39981329,  -53.147115  ,
         -130.51994586,  -19.14730614],
        [  -8.47873529,   -3.66034714,  -16.36526871,  -53.57795814,
         -131.37329743,  -20.33590886],
        [  -9.005292  ,   -3.72297329,  -16.69009443,  -52.83459743,
         -132.06858314,  -21.55787386],
        ..., 
        [   7.38921232,   -6.65697116,  -12.90909203,  -55.19439797,
          -76.15193725,    3.51976522],
        [  14.23744116,   -7.33555493,  -22.40803464,  -32.23130667,
          -72.34396464,   18.4534029 ],
        [  17.01390246,   -7.29852565,  -22.62766928,  -36.76503217,
          -71.06919623,   25.12335928]]),
 'start': 85.27,
 'v': 2.0}

In [185]:
data2[10]

{'a': 1.5,
 'd': 1.5,
 'emo_evo': [['sad'], ['sad', 'oth'], ['neu'], ['fru', 'sad']],
 'emotion': 'sad',
 'end': 32.2895,
 'id': 'Ses01F_impro02_F002',
 'mocap_head': array([[ -2.05221151,   3.01876619, -12.79437294,  -9.60066905,
         -14.7897196 , -33.16954889],
        [  5.90324278,   4.97932357,  -8.55396071, -15.43189556,
         -12.8678431 , -15.51001302],
        [ 18.19951222,   9.63267556,  -1.92307087, -26.66475548,
          -5.50974667,  14.08803921],
        ..., 
        [ 24.34034152,   1.30974544, -13.4797464 ,  16.3078488 ,
          22.81733032,  33.47179176],
        [ 23.63422768,   1.27257256, -13.24229936,  14.85766912,
          20.95039488,  31.07392616],
        [ 10.60502504,  -0.82678224, -13.01421016,  11.67028544,
          10.64773928,  -2.66802192]]),
 'start': 26.5659,
 'v': 2.0}

In [ ]:
Ses01F_impro01_F002

In [142]:
x_train2 = []
from sklearn.preprocessing import normalize

for ses_mod in data2:
    x = ses_mod['mocap_head']
    #x_normed = (x - x.min(0)) / x.ptp(0)
    #x_normed = x_normed - 0.5
    x_train2.append( x )
    
x_train2 = np.array(x_train2)
x_train2.shape

(9824, 200, 6)

In [143]:
x_train2 = x_train2.reshape(-1,1200)
x_train2.shape

(9824, 1200)

In [144]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = to_categorical(Y)

Y.shape

(9824, 4)

In [156]:
x_train2[2]

array([ -7.95062871,  -3.72252457, -16.39981329, ..., -36.76503217,
       -71.06919623,  25.12335928])

In [158]:
x_train2[15]

array([ -7.95062871,  -3.72252457, -16.39981329, ..., -36.76503217,
       -71.06919623,  25.12335928])

In [145]:
def build_simple2(nb_class, optimizer='SGD'):
    model = Sequential()
    model.add(Dense(512  , input_shape=(1200,)))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [146]:
model = build_simple2(nb_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               614912    
_________________________________________________________________
dropout_44 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               131328    
_________________________________________________________________
activation_44 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 1028      
_________________________________________________________________
activation_45 (Activation)   (None, 4)                 0         
Total para

In [147]:
hist = model.fit(x_train2, Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 7859 samples, validate on 1965 samples
Epoch 1/30
7859/7859 [==============================] - 1s 125us/step - loss: 1.4768 - acc: 0.3353 - val_loss: 1.3743 - val_acc: 0.3817
Epoch 2/30
7859/7859 [==============================] - 1s 70us/step - loss: 1.3733 - acc: 0.3382 - val_loss: 1.3673 - val_acc: 0.3817
Epoch 3/30
7859/7859 [==============================] - 1s 69us/step - loss: 1.3692 - acc: 0.3382 - val_loss: 1.3636 - val_acc: 0.3817
Epoch 4/30
7859/7859 [==============================] - 1s 69us/step - loss: 1.3672 - acc: 0.3382 - val_loss: 1.3615 - val_acc: 0.3817
Epoch 5/30
7859/7859 [==============================] - 1s 70us/step - loss: 1.3661 - acc: 0.3382 - val_loss: 1.3604 - val_acc: 0.3817
Epoch 6/30
7859/7859 [==============================] - 1s 70us/step - loss: 1.3656 - acc: 0.3382 - val_loss: 1.3601 - val_acc: 0.3817
Epoch 7/30
7859/7859 [==============================] - 1s 71us/step - loss: 1.3654 - acc: 0.3382 - val_loss: 1.3598 - val_acc: 0.3817
Epoch 

KeyboardInterrupt: 